In [1]:
%pip install polars

Note: you may need to restart the kernel to use updated packages.


In [2]:
import polars as pl
import subprocess


In [3]:
def build_and_prepare():
    cmd = """
    export CARGO_TARGET_DIR=/target
    export RUST_LOG=info
    cargo build
    """
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True, executable="/bin/bash")
    print(result)


def run_multiple_queries(start=1, stop=21, step=1):
    outputs = []
    for i in range(start, stop, step):
        outputs.append(run_single_query(i))
    return outputs


def run_single_query(query_number):
    cmd = f"""
    cd ../..
    export CARGO_TARGET_DIR=/target
    export RUST_LOG=warn
    cargo build
    cargo run --package reducer --bin reducer -- --query queries/query{query_number}/original_test.sql --test src/resources/native.sh
    """
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    output = result.stdout + result.stderr  # Capture both stdout and stderr if needed
    return output

##%%
def get_elements_by_pattern(elements: list[str], beginning, end) -> list[str]:
    orig_query_line = list(map(
        lambda s: s[s.index(beginning) + len(beginning) : s.index(end, s.index(beginning))],
        filter(
            lambda s: beginning in s and end in s and s.index(beginning) < s.index(end, s.index(beginning)),
            elements
        )
    ))
    
    # make sure exactly one output of this type can be found
    return orig_query_line



# Find parseerrors

In [4]:
build_and_prepare()
output = run_multiple_queries()
output

CompletedProcess(args='\n    export CARGO_TARGET_DIR=/target\n    export RUST_LOG=info\n    cargo build\n    ', returncode=0, stdout='', stderr='\x1b\x1b\x1bwarning\x1b\x1b\x1b: unused import: `std::collections::VecDeque`\x1b\n\x1b \x1b\x1b\x1b\x1b--> \x1b\x1bsrc/bin/unnest_select.rs:1:5\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b\x1b\x1b1\x1b\x1b \x1b\x1b\x1b\x1b|\x1b\x1b \x1b\x1buse std::collections::VecDeque;\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\x1b     \x1b\x1b\x1b\x1b^^^^^^^^^^^^^^^^^^^^^^^^^^\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b  \x1b\x1b\x1b\x1b= \x1b\x1b\x1bnote\x1b\x1b: `#[warn(unused_imports)]` on by default\x1b\n\n\x1b\x1b\x1bwarning\x1b\x1b\x1b: unused import: `std::sync::OnceLock`\x1b\n\x1b \x1b\x1b\x1b\x1b--> \x1b\x1bsrc/driver.rs:9:5\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b\x1b\x1b9\x1b\x1b \x1b\x1b\x1b\x1b|\x1b\x1b \x1b\x1buse std::sync::OnceLock;\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\x1b     \x1b\x1b\x1b\x1b^^^^^^^^^^^^^^^^^^^\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b  \x1b\x1b\x1b\x1b

['Current directory: /workspaces/reducer\n\x1b\x1b\x1bwarning\x1b\x1b\x1b: unused import: `std::collections::VecDeque`\x1b\n\x1b \x1b\x1b\x1b\x1b--> \x1b\x1bsrc/bin/unnest_select.rs:1:5\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b\x1b\x1b1\x1b\x1b \x1b\x1b\x1b\x1b|\x1b\x1b \x1b\x1buse std::collections::VecDeque;\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\x1b     \x1b\x1b\x1b\x1b^^^^^^^^^^^^^^^^^^^^^^^^^^\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b  \x1b\x1b\x1b\x1b= \x1b\x1b\x1bnote\x1b\x1b: `#[warn(unused_imports)]` on by default\x1b\n\n\x1b\x1b\x1bwarning\x1b\x1b\x1b: unused import: `std::sync::OnceLock`\x1b\n\x1b \x1b\x1b\x1b\x1b--> \x1b\x1bsrc/driver.rs:9:5\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b\x1b\x1b9\x1b\x1b \x1b\x1b\x1b\x1b|\x1b\x1b \x1b\x1buse std::sync::OnceLock;\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\x1b     \x1b\x1b\x1b\x1b^^^^^^^^^^^^^^^^^^^\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b  \x1b\x1b\x1b\x1b= \x1b\x1b\x1bnote\x1b\x1b: `#[warn(unused_imports)]` on by default\x1b\n\n\x1b\x1b\x1bwarning\x1b\x1b\

In [5]:
parsed_output = get_elements_by_pattern(output, "[ANALYSIS]", " [END ANALYSIS]")

In [6]:
parsed_output

[' "CREATE TABLE F (p BOOLEAN NOT NULL NULL NOT NULL, i BOOLEAN); INSERT INTO F SELECT * FROM (VALUES ((NOT false), false), (NULL, (NOT (NOT true)))) AS L WHERE (((+(+(-((+110) / (+((-(-150)) * ((247 * (91 * (-47))) + (-86)))))))) = ((((+(+(24 / (+((+89) * (+58)))))) * (-(-((193 + 223) / (-(222 / 219)))))) * (34 * 70)) * (+(+((((+(+(-202))) / (+52)) - (-(228 + (-104)))) * (-24)))))) = (false <> (66 <> 8)));"',
 ' "CREATE TABLE IF NOT EXISTS t_DX44 (c_LGUf NUMERIC, c_Hlmf3w REAL DEFAULT 749171.692897985, c_ewZ TEXT, c_EwP TEXT DEFAULT \'Fn58MvfLqzQ2DMC4\', c_YBA7sBV TEXT CHECK (length(c_YBA7sBV) > 0)); INSERT OR FAIL INTO t_DX44 (c_LGUf, c_Hlmf3w, c_ewZ, c_EwP, c_YBA7sBV) VALUES (-958347, 803354.0705377955, \'MQ_2\', \'qrZM84MTMHUkkov_3\', \'IcJ_4\'), (1119541, 661160.0780749931, \'7131k8CH2I7rflmaZmFh_102\', \'1sGjUivjzF_103\', \'fwAI_104\'), (2703615, 419682.84648422664, \'6u2sAbJVjXHWP_202\', \'YpYYmjS_203\', \'AyMTHlf_204\'); SELECT EXISTS (SELECT 1 FROM t_DX44 LIMIT 1) AS alias_xvE

In [7]:
s = ""
for i in parsed_output:
    s += i[2:-1]
s

'CREATE TABLE F (p BOOLEAN NOT NULL NULL NOT NULL, i BOOLEAN); INSERT INTO F SELECT * FROM (VALUES ((NOT false), false), (NULL, (NOT (NOT true)))) AS L WHERE (((+(+(-((+110) / (+((-(-150)) * ((247 * (91 * (-47))) + (-86)))))))) = ((((+(+(24 / (+((+89) * (+58)))))) * (-(-((193 + 223) / (-(222 / 219)))))) * (34 * 70)) * (+(+((((+(+(-202))) / (+52)) - (-(228 + (-104)))) * (-24)))))) = (false <> (66 <> 8)));CREATE TABLE IF NOT EXISTS t_DX44 (c_LGUf NUMERIC, c_Hlmf3w REAL DEFAULT 749171.692897985, c_ewZ TEXT, c_EwP TEXT DEFAULT \'Fn58MvfLqzQ2DMC4\', c_YBA7sBV TEXT CHECK (length(c_YBA7sBV) > 0)); INSERT OR FAIL INTO t_DX44 (c_LGUf, c_Hlmf3w, c_ewZ, c_EwP, c_YBA7sBV) VALUES (-958347, 803354.0705377955, \'MQ_2\', \'qrZM84MTMHUkkov_3\', \'IcJ_4\'), (1119541, 661160.0780749931, \'7131k8CH2I7rflmaZmFh_102\', \'1sGjUivjzF_103\', \'fwAI_104\'), (2703615, 419682.84648422664, \'6u2sAbJVjXHWP_202\', \'YpYYmjS_203\', \'AyMTHlf_204\'); SELECT EXISTS (SELECT 1 FROM t_DX44 LIMIT 1) AS alias_xvE FROM t_DX4

In [8]:
text_file = open("reduced.sql", "w")

text_file.write(s)

text_file.close()

In [9]:
statements = []
for i in parsed_output:
    
    statements.append(parsed_output[2:-1])

# Analyse reduction

In [10]:
title = ["orig-num-stmt","reduced-num-stmt","orig-token","reduced-token","time-taken"]
types = [pl.Int64, pl.Int64, pl.Int64, pl.Int64, pl.Float64]
empty_schema = {
    "query-num":  pl.Series(name="query-num", values=[], dtype=pl.Int32),
    **{
        title[i]: pl.Series(name=title[i], values=[], dtype=types[i])
        for i in range(len(title))
    }
}
master_df = pl.DataFrame(empty_schema)

In [11]:
master_df

query-num,orig-num-stmt,reduced-num-stmt,orig-token,reduced-token,time-taken
i32,i64,i64,i64,i64,f64


In [12]:
for i in range(1, 21):
    df = pl.read_csv(f"../output/result{i}.csv", separator=",", has_header=False, new_columns=title)
    df = df.with_columns(
        pl.lit(i).alias("query-num")
    )
    df = df.select(
    ["query-num"] + [col for col in df.columns if col != "query-num"])
    master_df = pl.concat([master_df, df], how="vertical")

In [13]:
master_df.head(20)

query-num,orig-num-stmt,reduced-num-stmt,orig-token,reduced-token,time-taken
i32,i64,i64,i64,i64,f64
1,2,2,74,74,118.853422
2,18,5,266,100,1024.436103
3,117,102,1002,1002,6116.207843
4,42,9,336,95,1983.151552
5,5,3,51,34,274.276597
…,…,…,…,…,…
16,2,2,3029,3029,110.761902
17,65,64,3754,3754,5921.78707
18,2,2,132,132,166.757995


In [14]:
master_df = master_df.with_columns(
    (100 - 100 / pl.col("orig-token") * pl.col("reduced-token"))
    .alias("pct")
)
master_df.head(20)

query-num,orig-num-stmt,reduced-num-stmt,orig-token,reduced-token,time-taken,pct
i32,i64,i64,i64,i64,f64,f64
1,2,2,74,74,118.853422,0.0
2,18,5,266,100,1024.436103,62.406015
3,117,102,1002,1002,6116.207843,0.0
4,42,9,336,95,1983.151552,71.72619
5,5,3,51,34,274.276597,33.333333
…,…,…,…,…,…,…
16,2,2,3029,3029,110.761902,1.4211e-14
17,65,64,3754,3754,5921.78707,0.0
18,2,2,132,132,166.757995,0.0


In [15]:
master_df[13]

query-num,orig-num-stmt,reduced-num-stmt,orig-token,reduced-token,time-taken,pct
i32,i64,i64,i64,i64,f64,f64
14,534,50,2631,403,26245.358616,84.68263
